In [2]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install tensorflow_io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import tensorflow_io as tfio
import numpy as np
import cv2
import pandas as pd
# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

keyValues = ['Nose', 'Left eye', 'Right eye', 'Left ear', 'Right ear', 'Left shoulder',
             'Right shoulder', 'Left elbow', 'Right elbow', 'Left wrist', 'Right wrist',
             'Left hip', 'Right hip', 'Left knee', 'Right knee', 'Left ankle', 'Right ankle']
tolerance = 30

In [6]:
#@title Helper functions for visualization

# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [ np.array(kpts_x), np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors
  

def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot
    
def to_gif(images, fps):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, fps=fps)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [7]:
model_name = "movenet_lightning" #@param ["movenet_lightning", "movenet_thunder", "movenet_lightning_f16.tflite", "movenet_thunder_f16.tflite", "movenet_lightning_int8.tflite", "movenet_thunder_int8.tflite"]

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    #print(keypoints_with_scores)
    return keypoints_with_scores

In [8]:
#@title Cropping Algorithm

# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inferece on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  #print(keypoints_with_scores)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) 
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1])
  return keypoints_with_scores

In [117]:
# Load the input image.
import random
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.int8)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, output_size = (225,225), frame_step = 1):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
  n_frames=int(video_length)
  # print(video_length)
  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

video_path = '/content/drive/MyDrive/EECE_5644_FINAL_PROJECT/data/test/curl/ORIGINAL/video2.mp4'

image = frames_from_video_file(video_path)
image.shape
#image = tf.io.read_file(image_path)
#print(image)
#image = tf.image.decode_image(image)
#image = tf.image.decode_gif(image)

(1313, 225, 225, 3)

In [118]:
def countRepetition(previous_pose, current_pose, previous_state, flag):
    # print(current_pose[0][0].shape)
    if current_pose[0][10][0] == 0 and current_pose[0][10][1] == 0:
        return 'Cannot detect any joint in the frame', previous_pose, previous_state, flag
    else:
        string, current_state = '', previous_state.copy()
        sdy,sdx = 0,0
        # Discard first 5 (0-4 indices) values, we don't need the value of nose, eye etc
        for i in range(5, 17):
            # The fancy text overlay
            string = string + keyValues[i] + ': '
            string = string + str('%.2f' % (current_pose[0][i][0])) + ' ' + str('%.2f' % current_pose[0][i][1]) + '\n'
            # If the difference is greater or lesser than tolerance or -tolerance sum it up or add 0 to sdx and sdy
            dx = (current_pose[0][i][0] - previous_pose[0][i][0])
            dy = (current_pose[0][i][1] - previous_pose[0][i][1])
            if(dx < tolerance and dx > (-1 * tolerance)):
                dx = 0
            if(dy < tolerance and dy > (-1 * tolerance)):
                dy = 0
            sdx += dx
            sdy += dy
        # if an overall average increase in value is detected set the current_state's bit to 1, if it decrease set it to 0
        # if it is between tolerance*3 and -tolerance*3, do nothing (then current_state will contain same value as previous)
        if(sdx > (tolerance*3)):
            current_state[0] = 1
        elif(sdx < (tolerance*-3)):
            current_state[0] = 0
        if(sdy > (tolerance*3)):
            current_state[1] = 1
        elif(sdy < (tolerance*-3)):
            current_state[1] = 0
        if(current_state != previous_state):
            flag = (flag + 1)%2
        return string, current_pose, current_state.copy(), flag

In [119]:
from google.colab.patches import cv2_imshow
# Load the input image.'

num_frames, image_height, image_width, _ = image.shape
crop_region = init_crop_region(image_height, image_width)
previous_pose = '' # '' denotes it is empty, really fast checking!
count = 0 # Stores the count of repetitions
        # A flag denoting change in state. 0 -> previous state is continuing, 1 -> state has changed
flag = -1
        # Novel string stores a pair of bits for each of the 12 joints denoting whether the joint is moving up or down
        # when plotted in a graph against time, 1 denotes upward and 0 denotes downward curving of the graph. It is initialised
        # as '22' so that current_state wont ever be equal to the string we generate unless there is no movement out of tolerance
current_state = [2,2]
output_images = []
keypoints=np.array([])


# num_frames=30
#bar = display(progress(0, num_frames-1), display_id=True)
print(num_frames)
for frame_idx in range(num_frames):
  keypoints_with_scores = run_inference(movenet, image[frame_idx, :, :, :], crop_region,crop_size=[input_size, input_size])
  # cv2_imshow(image[frame_idx, :, :, :].astype(np.int32))
  #print("random index is " + str(image[frame_idx, 200, 200, 1]))
  cv2_imshow(draw_prediction_on_image(image[frame_idx, :, :, :].astype(np.int32),keypoints_with_scores, crop_region=None,close_figure=True, output_image_height=image_height))
  # output_images.append(draw_prediction_on_image(image[frame_idx, :, :, :].astype(np.int32),keypoints_with_scores, crop_region=None,
      # close_figure=True, output_image_height=image_height))
  # crop_region = determine_crop_region(keypoints_with_scores, image_height, image_width)
  print(frame_idx)
  keypoints=np.append(keypoints,np.array([keypoints_with_scores[0][0][:,0],keypoints_with_scores[0][0][:,1]]))

  if(isinstance(previous_pose, str)): # if previous_pose was not inialised, assign the current keypoints to it
                previous_pose = keypoints_with_scores[0]
            
  text, previous_pose, current_state, flag = countRepetition(previous_pose, keypoints_with_scores[0], current_state, flag)

  if(flag == 1):
      count += 1
      flag = -1
      print("=======>         ",count,"    <===========")
  #bar.update(progress(frame_idx, num_frames-1))
keypoints=keypoints.reshape(num_frames,17*2).astype(int)
print("=======>         ",count,"    <===========")
# Prepare gif visualization.
# output = np.stack(output_images, axis=0)
# to_gif(output, fps=10)


Output hidden; open in https://colab.research.google.com to view.

In [109]:
print("=======>         ",count,"    <===========")

=======>          0     <===========


In [120]:

frame_count=0
feauters=17
cordinates=2
threshold_x=image_width*0.01
threshold_y=image_height*0.01
count=0
flag=0
blockMostvarianceIndex=-1


# print(threshold_x,threshold_y)
data=np.array([])

variance=np.array([])
meanv=np.array([])
changeinsequence=np.array([])
# print(start)
rep=np.array([])
for i in range(0,keypoints.shape[0]):
  frame_count+=1
  rep=np.append(rep,np.array(keypoints[i])).reshape(frame_count,feauters*cordinates)
  for j in range(0,feauters*cordinates):
          


          variance=np.append(variance,[rep[:,j].var()])
          # meanv=np.append(meanv,[rep[i,j]-rep[:,j].mean()])
          if(i>0):
              # print([-1 if keypoints[i,j,0]-keypoints[i,j,0]<0 else 1 , -1 if keypoints[i,j,1]-keypoints[i,j,1]<0 else 1])
              changeinsequence=np.append(changeinsequence,np.array([-1 if keypoints[i][j]-keypoints[i-1][j]<-2 else 1 ]))
  
  if(frame_count>10):
    variance=variance.reshape(i+1,feauters*cordinates)
    changeinsequence=changeinsequence.reshape(i,feauters*cordinates)
    if(blockMostvarianceIndex==-1):
        most_variance_indexes=np.argpartition(variance[-1], -4)[-4:]
    # print(most_variance_indexes[-1])
    if(((changeinsequence[-10:-5,most_variance_indexes[-1]].sum()<=-3 and changeinsequence[-5:,most_variance_indexes[-1]].sum()>=3)) 
    or ((changeinsequence[-10:-5,most_variance_indexes[-2]].sum()<=-3 and changeinsequence[-5:,most_variance_indexes[-2]].sum()>=3) )
    or ((changeinsequence[-10:-5,most_variance_indexes[-3]].sum()<=-3 and changeinsequence[-5:,most_variance_indexes[-3]].sum()>=3) )
    or ((changeinsequence[-10:-5,most_variance_indexes[-4]].sum()<=-3 and changeinsequence[-5:,most_variance_indexes[-4]].sum()>=3) )):
        if(flag==1):
          print("rep Match",frame_count,most_variance_indexes[-1],i)
          count+=1
        frame_count=0
        rep=np.array([])
        blockMostvarianceIndex=most_variance_indexes[-1]
        flag=-1
        # variance=np.array([])
        # changeinsequence=np.array([])
    if(( (changeinsequence[-10:-5,most_variance_indexes[-1]].sum()>=3 and changeinsequence[-5:,most_variance_indexes[-1]].sum()<=-3)) 
    or (changeinsequence[-10:-5,most_variance_indexes[-2]].sum()>=3 and changeinsequence[-5:,most_variance_indexes[-2]].sum()<=-3)
     or (changeinsequence[-10:-5,most_variance_indexes[-3]].sum()>=3 and changeinsequence[-5:,most_variance_indexes[-3]].sum()<=-3)
      or (changeinsequence[-10:-5,most_variance_indexes[-4]].sum()>=3 and changeinsequence[-5:,most_variance_indexes[-4]].sum()<=-3)):
        if(flag==-1):
          print("rep Match",frame_count,most_variance_indexes[-1],i)
          count+=1
        frame_count=0
        rep=np.array([])
        blockMostvarianceIndex=most_variance_indexes[-1]
        flag=1
        # variance=np.array([])
        # changeinsequence=np.array([])
    
            
variance=variance.reshape(keypoints.shape[0],feauters*cordinates)
changeinsequence=changeinsequence.reshape(keypoints.shape[0]-1,feauters*cordinates)
# meanv=meanv.reshape(keypoints.shape[0],feauters*cordinates)
print(changeinsequence,rep)  
pddaf1=pd.DataFrame(changeinsequence)
pddaf=pd.DataFrame(variance)
  # print(rep[:,0].mean(),rep[:,0,0])
  # print(rep)


rep Match 11 27 166
rep Match 105 27 271
rep Match 23 27 416
rep Match 109 27 525
rep Match 23 27 640
rep Match 192 27 832
rep Match 17 27 903
rep Match 117 27 1020
rep Match 20 27 1171
rep Match 99 27 1270
rep Match 26 27 1296
[[ 1.  1.  1. ...  1.  1.  1.]
 [ 1.  1.  1. ...  1.  1.  1.]
 [ 1.  1.  1. ...  1.  1.  1.]
 ...
 [ 1.  1.  1. ...  1.  1.  1.]
 [ 1.  1.  1. ...  1. -1.  1.]
 [ 1.  1.  1. ... -1.  1.  1.]] [[ 29.  17.  16.  10.  10.  56.  54. 123. 119.  72.  65. 168. 166. 219.
  223. 199. 214.  87.  93.  80. 111.  74. 139.  61. 152.  42. 178.  13.
  109.  66. 113.  57.  90.  77.]
 [ 29.  17.  16.  10.  10.  56.  54. 123. 119.  72.  65. 168. 166. 219.
  223. 199. 214.  87.  93.  80. 111.  74. 139.  61. 152.  42. 178.  13.
  109.  66. 113.  57.  90.  77.]
 [ 29.  17.  17.  10.  10.  56.  55. 124. 118.  72.  65. 168. 166. 219.
  223. 196. 214.  87.  93.  80. 111.  74. 138.  61. 152.  42. 179.  12.
  109.  67. 113.  58.  89.  77.]
 [ 28.  16.  16.  10.  12.  55.  55. 124. 119.  7

In [116]:
# pddaf1.head(60)
most_variance_indexes

array([26, 27, 10,  9])

In [27]:
np.argpartition(variance[-1], -4)[-4:]

array([26, 27,  9, 10])

In [23]:
pddaf.head(50)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.187500,0.187500,0.187500,0.750000,0.187500,0.187500,0.187500,0.000000,0.187500,0.000000,...,0.000000,0.187500,0.000000,0.000000,0.187500,0.187500,0.000000,0.000000,0.000000,1.687500
3,0.240000,0.640000,0.640000,0.960000,0.240000,0.240000,0.240000,0.000000,0.240000,0.000000,...,0.000000,0.240000,0.000000,0.000000,0.240000,0.240000,0.000000,0.000000,0.000000,2.160000
4,3.138889,1.333333,2.138889,1.000000,0.250000,0.250000,0.250000,0.000000,0.555556,0.138889,...,0.138889,0.250000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.138889,2.250000
5,2.857143,2.244898,3.061224,0.979592,0.244898,0.244898,0.489796,0.000000,0.530612,0.204082,...,0.204082,0.244898,0.122449,0.122449,0.244898,0.244898,0.000000,0.000000,0.204082,2.204082
6,2.609375,2.687500,3.484375,0.937500,0.234375,0.234375,0.609375,0.000000,0.500000,0.234375,...,0.234375,0.234375,0.187500,0.187500,0.234375,0.234375,0.000000,0.000000,0.234375,2.109375
7,2.395062,2.543210,3.283951,0.888889,0.222222,0.222222,0.543210,0.000000,0.469136,0.246914,...,0.246914,0.222222,0.469136,0.222222,0.395062,0.222222,0.395062,0.000000,0.246914,2.000000
8,2.160000,2.290000,2.960000,0.840000,1.200000,0.360000,0.490000,0.090000,0.450000,0.250000,...,0.250000,0.210000,0.640000,0.240000,0.490000,0.240000,0.640000,0.000000,0.250000,1.890000
9,1.966942,2.082645,2.694215,0.793388,1.834711,0.446281,0.446281,0.148760,0.429752,0.247934,...,0.247934,0.198347,0.743802,0.247934,0.545455,0.247934,0.793388,0.000000,0.247934,1.785124


In [99]:
print(count)


11
